In [1]:
import os
os.environ['FASTAI_TB_CLEAR_FRAMES']="1"

In [2]:
from fastai.text import * 

In [3]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/home/ubuntu/.fastai/data/imdb_sample')

In [4]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [ ]:
doc(lan)

In [5]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [6]:
learn = language_model_learner(data_lm, )
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.529614,4.015263,0.280357


In [7]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.168019,3.922085,0.288106


In [8]:
learn.save_encoder('ft_enc')

In [9]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(8866, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(8866, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1150, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1150, 1150, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1150, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=8866, bias=True)
    (output_dp): RNNDropout()
  )
)

In [10]:
learn.model[1].decoder.weight

Parameter containing:
tensor([[ 0.1500,  0.0511, -0.1269,  ...,  0.0057, -0.0233, -0.2449],
        [ 0.2079, -0.0437,  0.1241,  ..., -0.0269, -0.0769, -0.1537],
        [-0.7935,  0.1657,  0.0122,  ..., -0.3044, -0.0847, -0.0514],
        ...,
        [ 0.2525, -0.0366,  0.2212,  ..., -0.1012, -0.0522, -0.1254],
        [ 0.2671, -0.1452,  0.1510,  ..., -0.1040, -0.0185, -0.4618],
        [ 0.3107, -0.0108,  0.0955,  ...,  0.0359, -0.0326, -0.1716]],
       device='cuda:0', requires_grad=True)

### Clas

In [11]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [12]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.633422,0.646480,0.646766


In [13]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy
1,0.602290,0.560802,0.691542


In [14]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy
1,0.526755,0.521601,0.736318


In [15]:
learn.predict("This was a great movie!")

(Category positive, tensor(1), tensor([0.0325, 0.9675]))